## Django Template Tags and Filters

Django templates provide a powerful way to separate presentation logic from business logic. Template tags and filters allow you to manipulate data and control template rendering dynamically.

### What are Template Tags?
Template tags are functions that can be called from templates to perform complex operations, control flow, and generate content.

### What are Template Filters?
Template filters are simple functions that take a value and return a transformed version of that value.

### Key Differences
- **Tags**: Control template logic, loops, conditionals, etc.
- **Filters**: Transform values (strings, dates, numbers, etc.)

### Loading Tags and Filters
```django
{% load static %}
{% load custom_tags %}
{% load custom_filters %}
```

### Common Use Cases
- Formatting dates and numbers
- Conditional rendering
- Looping through data
- Including other templates
- Caching content
- Form rendering

## Built-in Template Tags

Django provides many built-in template tags for common operations. These tags control template logic, handle loops, manage template inheritance, and perform various utility functions.

### Control Flow Tags

#### if/elif/else
**Description**: The `if` tag evaluates a condition and renders content conditionally. It supports `elif` and `else` clauses for multiple conditions. The condition can use Django's template language operators and can check object attributes, boolean values, and use built-in tests.

**When to use**: Use for conditional rendering based on data values, user permissions, or application state.

**Operators available**: `and`, `or`, `not`, `==`, `!=`, `<`, `>`, `<=`, `>=`, `in`, `not in`, `is`, `is not`

```django
{% if user.is_authenticated %}
    <p>Welcome, {{ user.username }}!</p>
{% elif user.is_anonymous %}
    <p>Please log in.</p>
{% else %}
    <p>Something went wrong.</p>
{% endif %}
```

#### for loops
**Description**: The `for` tag iterates over sequences (lists, querysets, etc.) and renders content for each item. It supports an optional `empty` clause that renders when the sequence is empty. The tag provides access to loop variables through the `forloop` object.

**When to use**: Use to display lists of items, iterate through querysets, or repeat content for multiple data items.

**Features**: 
- Automatic handling of empty sequences
- Access to loop counters and state
- Support for nested loops

```django
{% for article in articles %}
    <h2>{{ article.title }}</h2>
    <p>{{ article.content|truncatewords:30 }}</p>
{% empty %}
    <p>No articles found.</p>
{% endfor %}

<!-- Loop variables example -->
{% for article in articles %}
    {{ forloop.counter }}. {{ article.title }}
    {% if not forloop.last %} | {% endif %}
{% endfor %}
```

#### forloop variables
**Description**: The `forloop` object provides information about the current loop iteration. These variables are only available within `for` loop blocks.

**Available variables**:
- `forloop.counter`: Current iteration (1-indexed)
- `forloop.counter0`: Current iteration (0-indexed) 
- `forloop.revcounter`: Reverse counter (counting down to 1)
- `forloop.revcounter0`: Reverse counter (counting down to 0)
- `forloop.first`: True for first iteration
- `forloop.last`: True for last iteration
- `forloop.parentloop`: Access parent loop variables in nested loops

### Template Inclusion Tags

#### include
**Description**: The `include` tag renders another template and inserts its output into the current template. It's useful for breaking large templates into smaller, reusable components. You can pass additional context variables using the `with` keyword.

**When to use**: Use for reusable template fragments like headers, footers, sidebars, or any repeated HTML blocks.

**Features**:
- Supports passing additional context
- Only passes the current context by default
- Useful for component-based template design

```django
<!-- Basic include -->
{% include "partials/header.html" %}

<!-- Include with additional context -->
{% include "partials/article.html" with article=article show_comments=True only %}
```

#### extends and block
**Description**: Template inheritance allows you to create a base template with common structure and override specific blocks in child templates. The `extends` tag specifies the parent template, while `block` tags define sections that can be overridden.

**When to use**: Use for consistent page layouts, shared headers/footers, and when you have multiple pages with similar structure but different content.

**Features**:
- Multiple inheritance levels
- Block nesting
- `{{ block.super }}` to include parent block content

```django
<!-- base.html -->
<!DOCTYPE html>
<html>
<head>
    <title>{% block title %}Default Title{% endblock %}</title>
    {% block extra_head %}{% endblock %}
</head>
<body>
    <header>{% block header %}Default Header{% endblock %}</header>
    <main>{% block content %}{% endblock %}</main>
    <footer>{% block footer %}Default Footer{% endblock %}</footer>
</body>
</html>

<!-- article.html -->
{% extends "base.html" %}

{% block title %}{{ article.title }}{% endblock %}

{% block extra_head %}
    <meta name="description" content="{{ article.excerpt }}">
{% endblock %}

{% block content %}
    <article>
        <h1>{{ article.title }}</h1>
        <div class="content">{{ article.content|safe }}</div>
    </article>
{% endblock %}
```

### Static Files and URLs

#### static
**Description**: The `static` tag generates the URL for static files (CSS, JavaScript, images). It automatically handles static file serving configuration and can prepend the STATIC_URL setting.

**When to use**: Always use for referencing static files instead of hardcoding URLs.

**Features**:
- Automatic URL generation
- Works with static file serving in development and production
- Supports static file versioning

```django
<!-- Basic static file reference -->
<img src="{% static 'images/logo.png' %}" alt="Logo">

<!-- Static files in CSS/JS -->
<link rel="stylesheet" href="{% static 'css/style.css' %}">
<script src="{% static 'js/app.js' %}"></script>

<!-- With dynamic filenames -->
<img src="{% static image_path %}" alt="Dynamic image">
```

#### url
**Description**: The `url` tag generates URLs for Django views based on URL pattern names. It's the template equivalent of Django's `reverse()` function and ensures URLs remain valid when URL patterns change.

**When to use**: Always use for internal links instead of hardcoding URLs.

**Features**:
- Supports URL parameters
- Handles URL namespaces
- Automatically updates when URLs change

```django
<!-- Basic URL generation -->
<a href="{% url 'article_list' %}">All Articles</a>

<!-- URL with parameters -->
<a href="{% url 'article_detail' article.id %}">Read More</a>
<a href="{% url 'category' category.slug %}">View Category</a>

<!-- URL with keyword arguments -->
<a href="{% url 'search' query=request.GET.q %}">Search Results</a>
```

### Form Rendering Tags

#### csrf_token
**Description**: The `csrf_token` tag generates a hidden input field with a CSRF token for protection against Cross-Site Request Forgery attacks. Django automatically validates this token on form submission.

**When to use**: Required in all POST forms when CSRF protection is enabled (which it is by default).

**Security**: Essential for preventing CSRF attacks by ensuring form submissions come from your site.

```django
<!-- Basic form with CSRF protection -->
<form method="post" action="{% url 'contact' %}">
    {% csrf_token %}
    <input type="text" name="name" placeholder="Your name">
    <input type="email" name="email" placeholder="Your email">
    <textarea name="message" placeholder="Your message"></textarea>
    <button type="submit">Send</button>
</form>

<!-- AJAX form (token in headers instead) -->
<form id="ajax-form" method="post">
    <input type="text" name="name">
    <button type="submit">Submit</button>
</form>
<script>
    // Include CSRF token in AJAX headers
    const csrftoken = document.querySelector('[name=csrfmiddlewaretoken]').value;
    // ... AJAX setup
</script>
```

### Caching Tags

#### cache
**Description**: The `cache` tag caches template fragments to improve performance. The cached content is stored with a given timeout and cache key. Multiple fragments can use the same cache key for coordinated invalidation.

**When to use**: Use for expensive template rendering, database-heavy sections, or content that doesn't change frequently.

**Features**:
- Configurable timeout
- Cache key variation based on context
- Automatic cache invalidation

```django
{% load cache %}

<!-- Basic caching for 5 minutes -->
{% cache 300 sidebar %}
    <div class="sidebar">
        {% for category in categories %}
            <li><a href="{% url 'category' category.slug %}">{{ category.name }}</a></li>
        {% endfor %}
    </div>
{% endcache %}

<!-- Cache with user-specific key -->
{% cache 600 user_menu request.user.pk %}
    <nav class="user-menu">
        <a href="{% url 'profile' %}">Profile</a>
        <a href="{% url 'settings' %}">Settings</a>
        {% if request.user.is_staff %}
            <a href="{% url 'admin' %}">Admin</a>
        {% endif %}
    </nav>
{% endcache %}

<!-- Cache with multiple key components -->
{% cache 3600 article_cache article.id request.user.is_authenticated %}
    <article>
        <h1>{{ article.title }}</h1>
        <div class="content">{{ article.content }}</div>
        {% if request.user.is_authenticated %}
            <div class="admin-controls">
                <a href="{% url 'edit_article' article.id %}">Edit</a>
            </div>
        {% endif %}
    </article>
{% endcache %}
```

### Other Useful Tags

#### with
**Description**: The `with` tag creates a local variable within a template block. It's useful for storing complex expressions or avoiding repeated computations.

**When to use**: Use to simplify complex expressions, avoid repeated calculations, or improve template readability.

```django
<!-- Store complex expression -->
{% with total=articles|length %}
    <p>Showing {{ total }} articles</p>
{% endwith %}

<!-- Multiple variables -->
{% with user_count=users|length article_count=articles|length %}
    <p>{{ user_count }} users, {{ article_count }} articles</p>
{% endwith %}

<!-- Avoid repeated computations -->
{% with featured=articles|first %}
    <div class="featured">
        <h2>{{ featured.title }}</h2>
        <p>{{ featured.excerpt }}</p>
    </div>
{% endwith %}
```

#### spaceless
**Description**: The `spaceless` tag removes whitespace between HTML tags. It only removes whitespace between tags, not within tag content or text nodes.

**When to use**: Use to minimize HTML output size, especially for production deployments where bandwidth matters.

**Note**: Only removes whitespace between tags, not within text content.

```django
{% spaceless %}
    <div>
        <h1>   Title   </h1>
        <p>
            Paragraph with
            multiple lines
        </p>
    </div>
{% endspaceless %}

<!-- Output (whitespace between tags removed) -->
<div><h1>   Title   </h1><p>
            Paragraph with
            multiple lines
        </p></div>
```

#### autoescape
**Description**: The `autoescape` tag controls Django's automatic HTML escaping. When set to `off`, content is not automatically escaped. When set to `on`, all variables are escaped.

**When to use**: 
- `autoescape off`: When rendering trusted HTML content
- `autoescape on`: Default behavior for security

**Security warning**: Only use `autoescape off` with trusted content to prevent XSS attacks.

```django
<!-- Default autoescaping (recommended) -->
<p>{{ user_input }}</p>  <!-- Escaped automatically -->

<!-- Disable autoescaping for trusted content -->
{% autoescape off %}
    <div>{{ trusted_html|safe }}</div>
{% endautoescape %}

<!-- Re-enable autoescaping -->
{% autoescape on %}
    <p>{{ user_input }}</p>  <!-- Escaped -->
{% endautoescape %}
```

#### comment
**Description**: The `comment` tag allows you to add comments to templates that won't appear in the rendered output. Unlike HTML comments, these are completely removed during template processing.

**When to use**: Use for template documentation, TODO notes, or temporarily disabling template code.

```django
<!-- This is an HTML comment (appears in output) -->

{% comment %}
    This is a template comment (removed from output)
    Can span multiple lines
    Useful for documentation
{% endcomment %}

<!-- Temporarily disable code -->
{% comment %}
    <div class="old-feature">
        This feature is deprecated
    </div>
{% endcomment %}
```

#### load
**Description**: The `load` tag imports custom template tags and filters from Django apps. It makes custom tags and filters available in the current template.

**When to use**: Required to use any custom template tags or filters in your templates.

```django
<!-- Load from single module -->
{% load custom_tags %}

<!-- Load multiple modules -->
{% load custom_tags custom_filters %}

<!-- Load specific tags/filters -->
{% load shout from custom_filters %}
{% load current_time multiply from custom_tags %}
```

#### firstof
**Description**: The `firstof` tag outputs the first argument that is not False. It's useful for providing fallback values or default content.

**When to use**: Use for displaying the first available value from a list of options, or providing defaults.

```django
<!-- Basic usage -->
{{ firstof article.title article.slug "Untitled" }}

<!-- With variables -->
{% firstof user.display_name user.username user.email "Anonymous" %}

<!-- Practical example -->
<img src="{% firstof article.image.url 'images/default.png' %}" alt="Article image">
```

#### regroup
**Description**: The `regroup` tag groups a list of objects by a common attribute. It's similar to SQL's GROUP BY clause and is useful for creating grouped displays.

**When to use**: Use to group lists of objects by a field value, such as grouping articles by publication date or products by category.

```django
<!-- Group articles by publication month -->
{% regroup articles by pub_date|date:"F Y" as article_groups %}

{% for group in article_groups %}
    <h2>{{ group.grouper }}</h2>
    <ul>
        {% for article in group.list %}
            <li><a href="{{ article.get_absolute_url }}">{{ article.title }}</a></li>
        {% endfor %}
    </ul>
{% endfor %}

<!-- Group by model field -->
{% regroup products by category.name as product_groups %}

{% for group in product_groups %}
    <h3>{{ group.grouper }}</h3>
    {% for product in group.list %}
        <div class="product">{{ product.name }}</div>
    {% endfor %}
{% endfor %}
```
    <h1>{{ article.title }}</h1>
    <p>{{ article.content }}</p>
{% endblock %}
```

### Static Files and URLs

#### static
```django
{% load static %}
<img src="{% static 'images/logo.png' %}" alt="Logo">
<link rel="stylesheet" href="{% static 'css/style.css' %}">
```

#### url
```django
<a href="{% url 'article_detail' article.id %}">Read more</a>
<form action="{% url 'search' %}" method="get">
```

### Form Rendering Tags

#### csrf_token
```django
<form method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Submit</button>
</form>
```

#### form field rendering
```django
{% for field in form %}
    <div class="field-wrapper {% if field.errors %}error{% endif %}">
        {{ field.label_tag }}
        {{ field }}
        {% if field.errors %}
            <ul class="errorlist">
                {% for error in field.errors %}
                    <li>{{ error }}</li>
                {% endfor %}
            </ul>
        {% endif %}
        {% if field.help_text %}
            <p class="help">{{ field.help_text }}</p>
        {% endif %}
    </div>
{% endfor %}
```

### Caching Tags

#### cache
```django
{% load cache %}
{% cache 300 sidebar request.user.username %}
    <!-- Expensive sidebar content -->
    {% for category in categories %}
        <li><a href="{% url 'category' category.slug %}">{{ category.name }}</a></li>
    {% endfor %}
{% endcache %}
```

### Other Useful Tags

#### with
```django
{% with total=articles|length %}
    <p>Showing {{ total }} articles</p>
{% endwith %}
```

#### spaceless
```django
{% spaceless %}
    <p>
        <a href="/">Home</a>
    </p>
{% endspaceless %}
```

#### autoescape
```django
{% autoescape off %}
    {{ content_with_html }}
{% endautoescape %}
```

## Built-in Template Filters

Template filters transform the value of a variable or argument. They are applied using the pipe (|) character and can be chained together.

### String Filters

#### Basic string manipulation
**Description**: These filters perform common string transformations like case conversion, whitespace handling, and text truncation.

**When to use**: Use for formatting text display, normalizing user input, or preparing content for presentation.

```django
{{ "hello world"|upper }}              <!-- HELLO WORLD -->
{{ "HELLO WORLD"|lower }}              <!-- hello world -->
{{ "hello world"|capfirst }}           <!-- Hello world -->
{{ "hello world"|title }}              <!-- Hello World -->
{{ "  hello  "|strip }}                <!-- hello -->
{{ "hello world"|truncatewords:2 }}    <!-- hello ... -->
{{ "hello world"|truncatechars:5 }}    <!-- he... -->
```

#### String replacement and formatting
**Description**: Filters for replacing text, padding strings, and formatting text layout.

**When to use**: Use for text substitution, creating fixed-width outputs, or aligning text in templates.

```django
{{ "Hello name"|replace:"name":"John" }}  <!-- Hello John -->
{{ "hello"|ljust:10 }}                     <!-- 'hello     ' -->
{{ "hello"|rjust:10 }}                     <!-- '     hello' -->
{{ "hello"|center:10 }}                    <!-- '  hello   ' -->
```

### Number Filters

#### Numeric formatting
**Description**: Filters for formatting numbers with commas, decimal places, and proper number display.

**When to use**: Use for displaying large numbers, currency values, or ensuring consistent number formatting across your site.

```django
{{ 1234567|intcomma }}         <!-- 1,234,567 -->
{{ 123.456|floatformat }}      <!-- 123.5 -->
{{ 123.456|floatformat:2 }}    <!-- 123.46 -->
{{ 123.456|floatformat:"0" }} <!-- 123 -->
```

#### Mathematical operations
**Description**: Filters for basic arithmetic and counting operations.

**When to use**: Use for simple calculations in templates or counting items in lists/querysets.

```django
{{ value|add:10 }}             <!-- value + 10 -->
{{ value|add:"-10" }}         <!-- value - 10 -->
{{ items|length }}             <!-- Length of list/queryset -->
```

### Date and Time Filters

#### Date formatting
**Description**: Filters for formatting date objects into human-readable strings using strftime-style format codes.

**When to use**: Use for displaying dates in various formats like "January 15, 2024" or "Jan 15, 2024".

```django
{{ article.pub_date|date:"F j, Y" }}     <!-- January 15, 2024 -->
{{ article.pub_date|date:"M d, Y" }}     <!-- Jan 15, 2024 -->
{{ article.pub_date|date:"D, M d" }}     <!-- Mon, Jan 15 -->
```

#### Time formatting
**Description**: Filters for formatting time portions of datetime objects.

**When to use**: Use for displaying time in 24-hour or 12-hour formats.

```django
{{ article.pub_date|time:"H:i" }}        <!-- 14:30 -->
{{ article.pub_date|time:"g:i A" }}      <!-- 2:30 PM -->
```

#### Relative time
**Description**: Filters that show the time elapsed since or until a date.

**When to use**: Use for showing "3 days ago" or "in 2 hours" style relative timestamps.

```django
{{ article.pub_date|timesince }}          <!-- 2 days, 3 hours -->
{{ article.pub_date|timeuntil }}          <!-- 2 days, 3 hours -->
```

### List and Sequence Filters

#### List operations
**Description**: Filters for accessing list elements and checking list properties.

**When to use**: Use for displaying first/last items, checking if lists are empty, or getting list lengths.

```django
{{ items|first }}                <!-- First item -->
{{ items|last }}                 <!-- Last item -->
{{ items|length }}               <!-- Number of items -->
{{ items|length_is:0 }}          <!-- True if empty -->
```

#### Slicing and selection
**Description**: Filters for extracting portions of lists or selecting random items.

**When to use**: Use for pagination displays, showing previews of lists, or random item selection.

```django
{{ items|slice:":3" }}          <!-- First 3 items -->
{{ items|slice:"1:4" }}         <!-- Items 1-3 -->
{{ items|random }}               <!-- Random item -->
```

### HTML and Escaping Filters

#### HTML escaping
**Description**: Filters for controlling HTML escaping and removing HTML tags.

**When to use**: 
- `escape`: Force escaping of potentially dangerous HTML
- `safe`: Mark content as safe to display without escaping
- `striptags`: Remove HTML tags to get plain text

```django
{{ content|escape }}             <!-- Escape HTML -->
{{ content|safe }}               <!-- Mark as safe HTML -->
{{ content|striptags }}          <!-- Remove HTML tags -->
```

#### URL encoding
**Description**: Filters for URL encoding and converting plain text URLs into clickable links.

**When to use**: Use for encoding URLs in query parameters or automatically linking URLs in text content.

```django
{{ url|urlencode }}              <!-- URL encode -->
{{ url|urlize }}                 <!-- Convert URLs to links -->
{{ text|urlizetrunc:30 }}        <!-- URLize with truncation -->
```

### Utility Filters

#### Default values
**Description**: Filters for providing fallback values when variables are empty or None.

**When to use**: Use for displaying default text when data is missing or empty.

```django
{{ value|default:"N/A" }}       <!-- Default if empty -->
{{ value|default_if_none:"N/A" }} <!-- Default if None -->
```

#### Conditional logic
**Description**: Filters that provide conditional text output based on values.

**When to use**: Use for yes/no displays, pluralization, or conditional text formatting.

```django
{{ value|yesno:"Yes,No,Maybe" }} <!-- Yes/No/Maybe based on truthiness -->
{{ items|pluralize }}           <!-- Add 's' if not singular -->
{{ items|pluralize:"ies,ies" }} <!-- Custom pluralization -->
```

#### JSON and data
**Description**: Filters for outputting data as JSON in script tags for JavaScript consumption.

**When to use**: Use for passing server data to client-side JavaScript in a safe, structured format.

```django
{{ data|json_script:"data" }}  <!-- Output as JSON script tag -->
```

### Chaining Filters
**Description**: Filters can be chained together using multiple pipe characters. They are applied from left to right.

**When to use**: Use for complex transformations that require multiple steps, like formatting and truncating text.

```django
{{ article.title|lower|capfirst|truncatewords:5 }}
{{ user.name|default:"Anonymous"|title }}
{{ price|floatformat:2|intcomma }}
```

## Custom Template Tags

Create custom template tags for complex logic that can't be handled with filters. Custom tags allow you to execute Python code and control template rendering dynamically.

### Setting Up Custom Tags

#### Create templatetags directory
**Description**: Django looks for custom template tags in `templatetags` directories within your apps. You need to create this directory and an `__init__.py` file to make it a Python package.

**When to use**: Required for any custom template tags in your Django project.

```python
# myapp/
#     templatetags/
#         __init__.py
#         custom_tags.py
```

#### Basic tag structure
**Description**: All custom template tags start with importing Django's template library and registering the tag. The `@register.simple_tag` decorator is the most common way to create custom tags.

**When to use**: Use this basic structure for all custom template tags.

```python
# templatetags/custom_tags.py
from django import template

register = template.Library()

@register.simple_tag
def current_time(format_string):
    from datetime import datetime
    return datetime.now().strftime(format_string)

# Usage in template
# {% load custom_tags %}
# {% current_time "%Y-%m-%d %H:%M:%S" %}
```

### Simple Tags

#### Basic simple tag
**Description**: Simple tags are the most common type of custom tag. They process arguments and return a string that gets inserted into the template. They're useful for calculations, formatting, or retrieving data.

**When to use**: Use for operations that return a single value to be displayed in the template.

```python
@register.simple_tag
def multiply(a, b):
    """Multiply two numbers"""
    return a * b

# Template usage
# {% multiply 5 3 %}  # Outputs: 15
```

#### Simple tag with context
**Description**: Simple tags can access the template context using the `takes_context=True` parameter. This allows them to access variables like `request`, `user`, and other context data.

**When to use**: Use when your tag needs access to template context variables like the current user or request object.

```python
@register.simple_tag(takes_context=True)
def get_user_avatar(context):
    user = context['user']
    if user.is_authenticated and user.profile.avatar:
        return user.profile.avatar.url
    return '/static/images/default-avatar.png'

# Template usage
# <img src="{% get_user_avatar %}" alt="Avatar">
```

### Inclusion Tags

#### Basic inclusion tag
**Description**: Inclusion tags render a separate template with the provided context and include the result in the current template. They're perfect for reusable UI components like pagination, menus, or widgets.

**When to use**: Use for complex HTML structures that are reused across templates, like navigation menus, pagination controls, or sidebar widgets.

```python
@register.inclusion_tag('partials/pagination.html')
def pagination(current_page, total_pages):
    """Render pagination controls"""
    return {
        'current_page': current_page,
        'total_pages': total_pages,
        'pages': range(1, total_pages + 1),
    }

# Template: partials/pagination.html
# <nav>
#     {% for page in pages %}
#         <a href="?page={{ page }}" {% if page == current_page %}class="active"{% endif %}>{{ page }}</a>
#     {% endfor %}
# </nav>

# Usage
# {% pagination current_page total_pages %}
```

#### Inclusion tag with context
**Description**: Inclusion tags can also access the current template context, allowing them to use existing context variables in addition to their own parameters.

**When to use**: Use when your included template needs access to the broader template context, not just the tag's specific parameters.

```python
@register.inclusion_tag('partials/user_menu.html', takes_context=True)
def user_menu(context):
    user = context['user']
    return {
        'user': user,
        'menu_items': [
            {'url': 'profile', 'text': 'Profile'},
            {'url': 'settings', 'text': 'Settings'},
            {'url': 'logout', 'text': 'Logout'},
        ]
    }
```

### Assignment Tags

#### Assignment tag (Django < 1.9)
**Description**: Assignment tags store the result of a computation in a template variable for later use. In Django 1.9+, simple tags can be used with the `as` keyword instead.

**When to use**: Use to store complex query results or computed values in template variables for reuse throughout the template.

```python
@register.assignment_tag
def get_featured_articles():
    return Article.objects.filter(featured=True)[:5]

# Usage
# {% get_featured_articles as articles %}
# {% for article in articles %}
#     <h3>{{ article.title }}</h3>
# {% endfor %}
```

### Complex Tags with Parsing

#### Custom tag with arguments
**Description**: For complex logic, you can create custom tags that parse their own arguments and create Node objects. This gives you full control over template parsing and rendering.

**When to use**: Use for advanced template logic that needs custom parsing, like conditional blocks, loops, or complex data manipulation.

```python
from django.template import Node

class RecentArticlesNode(Node):
    def __init__(self, limit, var_name):
        self.limit = limit
        self.var_name = var_name
    
    def render(self, context):
        articles = Article.objects.all()[:self.limit]
        context[self.var_name] = articles
        return ''

@register.tag
def get_recent_articles(parser, token):
    """Usage: {% get_recent_articles 5 as recent_articles %}"""
    bits = token.split_contents()
    if len(bits) != 4 or bits[2] != 'as':
        raise template.TemplateSyntaxError(
            "get_recent_articles tag requires format: "
            "{% get_recent_articles limit as variable_name %}"
        )
    limit = bits[1]
    var_name = bits[3]
    return RecentArticlesNode(limit, var_name)

# Usage
# {% get_recent_articles 10 as articles %}
# {% for article in articles %}{{ article.title }}{% endfor %}
```

### Block Tags

#### Custom block tag
**Description**: Block tags can contain other template content and control how that content is rendered. They're used for caching, conditional rendering, and other complex template logic.

**When to use**: Use for operations that need to wrap or modify blocks of template content, like caching, permissions, or custom control structures.

```python
class CacheNode(Node):
    def __init__(self, nodelist, key):
        self.nodelist = nodelist
        self.key = key
    
    def render(self, context):
        from django.core.cache import cache
        
        cache_key = self.key.resolve(context)
        content = cache.get(cache_key)
        
        if content is None:
            content = self.nodelist.render(context)
            cache.set(cache_key, content, 300)  # 5 minutes
        
        return content

@register.tag
def custom_cache(parser, token):
    """Custom caching tag: {% custom_cache "key" %}...{% endcustom_cache %}"""
    bits = token.split_contents()
    if len(bits) != 2:
        raise template.TemplateSyntaxError(
            "custom_cache tag requires one argument: the cache key"
        )
    
    key = parser.compile_filter(bits[1])
    nodelist = parser.parse(('endcustom_cache',))
    parser.delete_first_token()
    return CacheNode(nodelist, key)

# Usage
# {% custom_cache "sidebar" %}
#     <!-- Expensive content here -->
# {% endcustom_cache %}
```

## Custom Template Filters

Create custom filters for data transformation that can be chained with other filters.

### Basic Custom Filters

#### Simple filter
```python
# templatetags/custom_filters.py
from django import template
from django.template.defaultfilters import stringfilter

register = template.Library()

@register.filter
@stringfilter
def shout(value):
    """Convert text to uppercase and add exclamation marks"""
    return f"{value.upper()}!!!"

# Usage
# {{ "hello world"|shout }}  # Outputs: HELLO WORLD!!!
```

#### Filter with arguments
```python
@register.filter
def truncate_chars(value, arg):
    """Truncate text to specified number of characters"""
    try:
        length = int(arg)
    except ValueError:
        return value
    
    if len(value) <= length:
        return value
    
    return value[:length] + '...'

# Usage
# {{ "This is a long text"|truncate_chars:10 }}  # Outputs: This is a ...
```

### Advanced Custom Filters

#### Filter that needs auto-escaping
```python
@register.filter
def highlight(search_term, text):
    """Highlight search term in text"""
    from django.utils.safestring import mark_safe
    import re
    
    if not search_term:
        return text
    
    highlighted = re.sub(
        f'({re.escape(search_term)})',
        r'<mark>\1</mark>',
        str(text),
        flags=re.IGNORECASE
    )
    return mark_safe(highlighted)

# Usage
# {{ article.content|highlight:request.GET.search }}
```

#### Filter that returns HTML safely
```python
@register.filter
def star_rating(rating):
    """Convert numeric rating to star display"""
    from django.utils.safestring import mark_safe
    
    try:
        rating = int(rating)
    except (ValueError, TypeError):
        return ''
    
    full_stars = '★' * rating
    empty_stars = '☆' * (5 - rating)
    
    return mark_safe(f'<span class="rating">{full_stars}{empty_stars}</span>')

# Usage
# {{ product.rating|star_rating }}
```

### Filters for Model Objects

#### Filter for model relationships
```python
@register.filter
def get_related(model_obj, field_name):
    """Get related objects dynamically"""
    try:
        related_manager = getattr(model_obj, field_name)
        if hasattr(related_manager, 'all'):
            return related_manager.all()
        return related_manager
    except AttributeError:
        return None

# Usage
# {{ article|get_related:"tags"|length }} tags

```
This gets the related 'tags' for an article and counts them.

#### Filter for model methods
```python
@register.filter
def call_method(obj, method_name):
    """Call a method on an object"""
    try:
        method = getattr(obj, method_name)
        if callable(method):
            return method()
        return method
    except AttributeError:
        return None

# Usage
# {{ article|call_method:"get_absolute_url" }}

```
This calls the 'get_absolute_url' method on the article object. This is useful for dynamically invoking methods in templates.

### Date and Time Filters

#### Custom date formatting
```python
@register.filter
def relative_date(value):
    """Return relative date string"""
    from django.utils.timesince import timesince
    from django.utils import timezone
    import datetime
    
    if not value:
        return ''
    
    now = timezone.now()
    
    if isinstance(value, datetime.datetime):
        diff = now - value
        if diff.days == 0:
            return "Today"
        elif diff.days == 1:
            return "Yesterday"
        elif diff.days < 7:
            return f"{diff.days} days ago"
        else:
            return value.strftime("%B %d, %Y")
    
    return value

# Usage
# {{ article.pub_date|relative_date }}

```
This displays a human-friendly relative date. This is useful for blog posts or news articles where you want to show how recent the content is.

### Utility Filters

#### Filter for debugging
```python
@register.filter
def debug(value):
    """Debug filter - prints value to console"""
    print(f"DEBUG: {value} (type: {type(value)})")
    return value

# Usage (remove in production)
# {{ some_var|debug }}
```


#### Filter for conditional classes
```python
@register.filter
def add_class(value, css_class):
    """Add CSS class to form field"""
    if hasattr(value, 'field'):
        # Django form field
        value.field.widget.attrs['class'] = css_class
        return value
    
    # Regular string
    return f'{value} {css_class}'.strip()

# Usage
# {{ form.field|add_class:"form-control" }}
```

### Filter Registration

#### Registering filters with custom names
```python
# Register with custom name
register.filter('uppercase_shout', shout)

# Register with different name than function
@register.filter(name='excerpt')
def get_excerpt(text):
    """Get first 100 characters as excerpt"""
    if len(text) <= 100:
        return text
    return text[:100] + '...'

# Usage
# {{ article.content|excerpt }}
```

#### Filter that expects localtime
```python
@register.filter(expects_localtime=True)
def business_hours(value):
    """Check if datetime is during business hours"""
    if hasattr(value, 'hour'):
        return 9 <= value.hour < 17
    return False

# Usage
{% if event.start_time|business_hours %}During business hours{% endif %}
```


## Advanced Template Techniques

Advanced template techniques help you build more maintainable, efficient, and feature-rich Django applications. These patterns go beyond basic template usage.

### Template Inheritance Patterns

#### Multiple inheritance
**Description**: Django supports multiple levels of template inheritance, allowing you to create intermediate base templates that extend the root base template. This creates a hierarchy of templates with increasing specificity.

**When to use**: Use for complex sites with multiple page types that share common elements but have different layouts or sections.

```django
<!-- base.html -->
<!DOCTYPE html>
<html>
<head>
    {% block head %}
        <title>{% block title %}Default Title{% endblock %}</title>
    {% endblock %}
</head>
<body>
    {% block content %}{% endblock %}
</body>
</html>

<!-- article_base.html -->
{% extends "base.html" %}

{% block head %}
    {{ block.super }}
    <meta name="description" content="Article about {{ article.title }}">
{% endblock %}

<!-- article_detail.html -->
{% extends "article_base.html" %}

{% block title %}{{ article.title }}{% endblock %}

{% block content %}
    <article>
        <h1>{{ article.title }}</h1>
        <p class="meta">By {{ article.author }} on {{ article.pub_date|date }}</p>
        <div class="content">{{ article.content|safe }}</div>
    </article>
{% endblock %}
```

### Context Processors

#### Creating context processors
**Description**: Context processors are functions that add variables to the template context automatically for every template. They're useful for global data like site settings, user information, or navigation menus.

**When to use**: Use for data that needs to be available in all or most templates, such as site-wide settings, user authentication status, or common navigation elements.

```python
# context_processors.py
def site_settings(request):
    """Add site settings to all templates"""
    from .models import SiteSettings
    
    try:
        settings = SiteSettings.objects.get(pk=1)
    except SiteSettings.DoesNotExist:
        settings = None
    
    return {
        'site_settings': settings,
        'current_path': request.path,
    }

# settings.py
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
                'myapp.context_processors.site_settings',
            ],
        },
    },
]

# Template usage
# {{ site_settings.site_name }}
# {{ current_path }}
```

### Template Loaders

#### Custom template loader
**Description**: Custom template loaders allow you to load templates from non-standard locations or with custom logic. This is useful for theming, multi-tenant applications, or custom template storage.

**When to use**: Use for applications that need dynamic template loading, theming support, or templates stored in custom locations like databases or remote storage.

```python
# template_loaders.py
from django.template.loaders.filesystem import Loader as FilesystemLoader
from django.template import Origin
import os

class ThemeLoader(FilesystemLoader):
    """Load templates from theme directories"""
    
    def get_template_sources(self, template_name):
        # Check theme directory first
        theme_dir = getattr(self, 'theme_dir', 'default_theme')
        theme_path = os.path.join('themes', theme_dir, template_name)
        
        for path in self.path:
            yield Origin(
                self.path,
                os.path.join(path, theme_path),
                self,
                template_name
            )
        
        # Fall back to default paths
        yield from super().get_template_sources(template_name)

# settings.py
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': ['templates'],
        'APP_DIRS': True,
        'OPTIONS': {
            'loaders': [
                ('myapp.template_loaders.ThemeLoader', ['templates']),
                'django.template.loaders.filesystem.Loader',
                'django.template.loaders.app_directories.Loader',
            ],
        },
    },
]
```

### Template Response Mixin

#### Using TemplateResponseMixin
**Description**: TemplateResponse allows you to modify the HTTP response after template rendering but before it's sent to the client. This is useful for adding headers, post-processing content, or implementing custom caching.

**When to use**: Use when you need to modify response headers, add post-rendering callbacks, or implement custom response processing.

```python
# views.py
from django.views.generic import TemplateView
from django.template.response import TemplateResponse

class ArticleView(TemplateView):
    template_name = 'article.html'
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['article'] = Article.objects.get(slug=kwargs['slug'])
        return context
    
    def render_to_response(self, context, **response_kwargs):
        # Custom rendering logic
        response = super().render_to_response(context, **response_kwargs)
        
        # Add custom headers
        response['X-Article-ID'] = context['article'].id
        
        return response

# Or use TemplateResponse directly
def article_view(request, slug):
    article = Article.objects.get(slug=slug)
    
    # TemplateResponse allows post-processing
    response = TemplateResponse(
        request,
        'article.html',
        {'article': article}
    )
    
    # Modify response after template rendering
    response.add_post_render_callback(lambda r: post_process_article(r, article))
    
    return response
```

### Template Fragments and AJAX

#### Template fragments for AJAX
**Description**: Template fragments allow you to render partial templates for AJAX requests, enabling dynamic content updates without full page reloads. This works well with libraries like HTMX or vanilla JavaScript.

**When to use**: Use for single-page application features, dynamic content loading, or progressive enhancement of traditional web applications.

```django
<!-- article_list.html -->
{% for article in articles %}
    {% include "partials/article_item.html" with article=article %}
{% endfor %}

<!-- partials/article_item.html -->
<article class="article-item" data-id="{{ article.id }}">
    <h3>{{ article.title }}</h3>
    <p>{{ article.excerpt }}</p>
    <time>{{ article.pub_date|date }}</time>
</article>

# views.py
def article_list_ajax(request):
    articles = Article.objects.all()[:10]
    
    if request.headers.get('HX-Request'):  # HTMX request
        return render(request, 'partials/article_item.html', {
            'article': articles[0]  # Return single item for HTMX
        })
    
    return render(request, 'article_list.html', {
        'articles': articles
    })
```

### Template Debugging

#### Debug template context
**Description**: Debug context processors add debugging information to templates during development. This helps you inspect request data, user information, and session variables.

**When to use**: Use during development to debug template context, inspect request data, or troubleshoot template rendering issues.

```python
# context_processors.py
def debug_context(request):
    """Add debug information to context"""
    return {
        'debug': True,
        'request': request,
        'user': request.user,
        'session': dict(request.session),
    }

# Template debugging
# {% if debug %}
#     <div class="debug-info">
#         <h4>Debug Information</h4>
#         <pre>{{ request.META|json_script:"meta" }}</pre>
#         <pre>{{ session|json_script:"session" }}</pre>
#     </div>
# {% endif %}
```

#### Template error handling
**Description**: Custom filters and tags can provide safe fallbacks when data is missing or invalid, preventing template errors from breaking page rendering.

**When to use**: Use to make templates more robust by gracefully handling missing attributes, invalid data, or unexpected None values.

```python
# Custom template tag for safe attribute access
@register.filter
def get_attr(obj, attr_name):
    """Safely get attribute from object"""
    try:
        return getattr(obj, attr_name, '')
    except:
        return ''

# Usage
# {{ object|get_attr:"potentially_missing_attr"|default:"N/A" }}
```

## Best Practices for Template Tags and Filters

### 1. Naming Conventions
- Use lowercase with underscores for filter names
- Use descriptive names that indicate functionality
- Avoid conflicts with built-in tags/filters

### 2. Error Handling
```python
@register.filter
def safe_divide(value, arg):
    """Safely divide two numbers"""
    try:
        return float(value) / float(arg)
    except (ValueError, TypeError, ZeroDivisionError):
        return 0

@register.simple_tag
def get_object_or_none(model_name, object_id):
    """Get model object or None if not found"""
    try:
        model = apps.get_model(model_name)
        return model.objects.get(pk=object_id)
    except (LookupError, ValueError):
        return None
```

### 3. Performance Considerations
- Cache expensive operations in custom tags
- Use select_related/prefetch_related in model queries
- Avoid database queries in filters (use tags instead)
- Minimize template rendering time

### 4. Security Best Practices
- Always escape output unless marked as safe
- Validate input parameters
- Use Django's ORM safely
- Avoid executing arbitrary code

### 5. Documentation
```python
@register.filter
def format_currency(value, currency='USD'):
    """
    Format a number as currency.
    
    Args:
        value: Numeric value to format
        currency: Currency code (USD, EUR, etc.)
    
    Returns:
        Formatted currency string
    
    Example:
        {{ 1234.56|format_currency:"USD" }} -> "$1,234.56"
    """
    # Implementation here
    pass
```

### 6. Testing Custom Tags and Filters
- Test with various input types
- Test edge cases and error conditions
- Test template rendering
- Mock external dependencies

### 7. Organization
- Group related tags/filters in same module
- Use clear module names (forms_tags.py, date_filters.py)
- Keep complex logic in separate utilities

### 8. Django Version Compatibility
- Check for deprecated features
- Use appropriate decorators
- Test with target Django versions

### 9. Common Pitfalls to Avoid
1. **Database queries in filters**: Filters should be pure functions
2. **Complex logic in templates**: Move logic to views or custom tags
3. **Not handling None values**: Always check for None/empty values
4. **Memory leaks**: Don't store large objects in template context
5. **Thread safety**: Be careful with shared state

### 10. Advanced Patterns

#### Lazy evaluation
```python
class LazyObject:
    """Lazy evaluation for expensive operations"""
    def __init__(self, func, *args, **kwargs):
        self.func = func
        self.args = args
        self.kwargs = kwargs
        self._value = None
    
    def __str__(self):
        if self._value is None:
            self._value = self.func(*self.args, **self.kwargs)
        return str(self._value)

@register.simple_tag
def lazy_expensive_operation():
    """Only execute when actually rendered"""
    return LazyObject(expensive_database_query)
```

#### Context-aware tags
```python
@register.simple_tag(takes_context=True)
def user_can_edit(context, obj):
    """Check if user can edit object based on permissions"""
    user = context['user']
    return user.has_perm(f'app.change_{obj.__class__.__name__.lower()}', obj)

# Usage
# {% if user_can_edit article %}
#     <a href="{% url 'edit_article' article.id %}">Edit</a>
# {% endif %}
```

## Testing Template Tags and Filters

### Testing Custom Filters

#### Basic filter testing
```python
# tests.py
from django.test import TestCase
from django.template import Template, Context
from myapp.templatetags.custom_filters import shout

class CustomFilterTest(TestCase):
    def test_shout_filter(self):
        """Test the shout filter"""
        # Test direct function call
        result = shout("hello world")
        self.assertEqual(result, "HELLO WORLD!!!")
    
    def test_shout_filter_in_template(self):
        """Test filter in template context"""
        template = Template('{% load custom_filters %}{{ text|shout }}')
        context = Context({'text': 'hello world'})
        result = template.render(context)
        self.assertEqual(result, 'HELLO WORLD!!!')
    
    def test_shout_filter_edge_cases(self):
        """Test filter with edge cases"""
        # Empty string
        self.assertEqual(shout(''), '!!!')
        
        # None value
        self.assertEqual(shout(None), 'NONE!!!')
        
        # Numbers
        self.assertEqual(shout(123), '123!!!')
```

#### Testing filters with arguments
```python
class TruncateFilterTest(TestCase):
    def test_truncate_chars_basic(self):
        """Test basic truncation"""
        result = truncate_chars("Hello World", 5)
        self.assertEqual(result, "Hello...")
    
    def test_truncate_chars_no_truncation(self):
        """Test when no truncation is needed"""
        result = truncate_chars("Hello", 10)
        self.assertEqual(result, "Hello")
    
    def test_truncate_chars_invalid_arg(self):
        """Test with invalid argument"""
        result = truncate_chars("Hello World", "invalid")
        self.assertEqual(result, "Hello World")
```

### Testing Custom Tags

#### Testing simple tags
```python
class SimpleTagTest(TestCase):
    def test_current_time_tag(self):
        """Test current_time simple tag"""
        template = Template('{% load custom_tags %}{% current_time "%Y-%m-%d" %}')
        context = Context({})
        result = template.render(context)
        
        # Check that result matches expected date format
        from datetime import datetime
        expected = datetime.now().strftime("%Y-%m-%d")
        self.assertEqual(result, expected)
    
    def test_multiply_tag(self):
        """Test multiply tag with arguments"""
        template = Template('{% load custom_tags %}{% multiply 5 3 %}')
        context = Context({})
        result = template.render(context)
        self.assertEqual(result, '15')
```

#### Testing inclusion tags
```python
class InclusionTagTest(TestCase):
    def test_pagination_tag(self):
        """Test pagination inclusion tag"""
        template = Template('{% load custom_tags %}{% pagination 2 5 %}')
        context = Context({})
        result = template.render(context)
        
        # Check that pagination HTML is rendered
        self.assertIn('current_page', result)
        self.assertIn('total_pages', result)
        self.assertIn('pages', result)
    
    def test_pagination_context(self):
        """Test that pagination tag provides correct context"""
        from myapp.templatetags.custom_tags import pagination
        
        context_dict = pagination(3, 10)
        self.assertEqual(context_dict['current_page'], 3)
        self.assertEqual(context_dict['total_pages'], 10)
        self.assertEqual(list(context_dict['pages']), list(range(1, 11)))
```

### Testing Complex Tags

#### Testing assignment tags
```python
class AssignmentTagTest(TestCase):
    def test_get_featured_articles(self):
        """Test assignment tag"""
        template = Template(
            '{% load custom_tags %}'
            '{% get_featured_articles as articles %}'
            '{{ articles|length }}'
        )
        context = Context({})
        result = template.render(context)
        
        # Should have articles in context
        self.assertIsNotNone(context.get('articles'))
        self.assertIn('5', result)  # Assuming 5 featured articles
```

#### Testing block tags
```python
class BlockTagTest(TestCase):
    def test_custom_cache_tag(self):
        """Test custom cache block tag"""
        template = Template(
            '{% load custom_tags %}'
            '{% custom_cache "test_key" %}'
            'Expensive content: {{ counter }}'
            '{% endcustom_cache %}'
        )
        
        # First render
        context1 = Context({'counter': 1})
        result1 = template.render(context1)
        
        # Second render with different context
        context2 = Context({'counter': 2})
        result2 = template.render(context2)
        
        # Should return cached result
        self.assertEqual(result1, result2)
        self.assertIn('Expensive content: 1', result1)
```

### Testing Template Context

#### Testing context processors
```python
class ContextProcessorTest(TestCase):
    def test_site_settings_context_processor(self):
        """Test context processor adds settings"""
        from django.test import RequestFactory
        from myapp.context_processors import site_settings
        
        factory = RequestFactory()
        request = factory.get('/')
        
        context = site_settings(request)
        self.assertIn('site_settings', context)
        self.assertIn('current_path', context)
        self.assertEqual(context['current_path'], '/')
```

### Integration Testing

#### Testing templates with custom tags/filters
```python
class TemplateIntegrationTest(TestCase):
    def test_article_template_renders_correctly(self):
        """Test complete template rendering with custom elements"""
        article = Article.objects.create(
            title="Test Article",
            content="Test content",
            pub_date=timezone.now()
        )
        
        template = Template(
            '{% load custom_filters custom_tags %}'
            '<article>'
            '    <h1>{{ article.title|shout }}</h1>'
            '    <time>{{ article.pub_date|relative_date }}</time>'
            '    {% get_related article "tags" as tags %}' 
            '    <div class="tags">'
            '        {% for tag in tags %}' 
            '            <span class="tag">{{ tag.name }}</span>'
            '        {% endfor %}' 
            '    </div>'
            '</article>'
        )
        
        context = Context({'article': article})
        result = template.render(context)
        
        self.assertIn('TEST ARTICLE!!!', result)
        self.assertIn('<article>', result)
        self.assertIn('</article>', result)
```

### Mocking and Fixtures

#### Using fixtures for template testing
```python
class TemplateFixtureTest(TestCase):
    fixtures = ['articles.json', 'iam.json']
    
    def test_template_with_fixtures(self):
        """Test template rendering with fixture data"""
        article = Article.objects.get(pk=1)
        
        response = self.client.get(f'/article/{article.slug}/')
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, article.title)
        
        # Test custom filter in rendered template
        self.assertContains(response, 'TEST ARTICLE!!!')  # Assuming shout filter
```

#### Mocking external dependencies
```python
from unittest.mock import patch, MagicMock

class MockedTemplateTest(TestCase):
    @patch('myapp.templatetags.custom_tags.send_email')
    def test_email_tag_with_mock(self, mock_send):
        """Test email tag with mocked email sending"""
        mock_send.return_value = True
        
        template = Template('{% load custom_tags %}{% send_email "test@example.com" %}')
        context = Context({})
        result = template.render(context)
        
        mock_send.assert_called_once_with('test@example.com')
        self.assertIn('Email sent', result)
```

### Performance Testing

#### Testing template rendering performance
```python
import time
from django.test.utils import override_settings

class TemplatePerformanceTest(TestCase):
    def test_template_rendering_speed(self):
        """Test that template renders within acceptable time"""
        articles = Article.objects.all()[:100]  # Large queryset
        
        template = Template(
            '{% for article in articles %}' 
            '    <div>{{ article.title|truncate_chars:50 }}</div>'
            '{% endfor %}'
        )
        
        context = Context({'articles': articles})
        
        start_time = time.time()
        result = template.render(context)
        end_time = time.time()
        
        rendering_time = end_time - start_time
        self.assertLess(rendering_time, 1.0)  # Should render in less than 1 second
        self.assertIn('div', result)
```

### Error Testing

#### Testing error conditions
```python
class TemplateErrorTest(TestCase):
    def test_filter_handles_errors_gracefully(self):
        """Test that filters handle errors without breaking template"""
        template = Template('{{ value|safe_divide:0 }}')
        
        # Test with None
        context = Context({'value': None})
        result = template.render(context)
        self.assertEqual(result, '0')
        
        # Test with invalid value
        context = Context({'value': 'invalid'})
        result = template.render(context)
        self.assertEqual(result, '0')
    
    def test_tag_handles_missing_objects(self):
        """Test tags handle missing objects gracefully"""
        template = Template('{% get_object_or_none "nonexistent" 999 %}')
        context = Context({})
        result = template.render(context)
        self.assertEqual(result, '')  # Should render nothing
```

## Performance Considerations and Monitoring

Optimizing template performance is crucial for maintaining fast response times and good user experience. Django templates can become performance bottlenecks if not carefully managed.

### Template Rendering Optimization

**Description**: Template rendering optimization focuses on reducing the time spent processing templates and minimizing database queries triggered during rendering.

**Key strategies**: Pre-load related data, cache expensive operations, and avoid complex computations in templates.

#### 1. Minimize Database Queries in Templates
**Description**: Database queries in templates, especially in loops, can cause N+1 query problems where one query per item is executed.

**When to avoid**: Never access related model fields in template loops without pre-loading the relationships.

```python
# Bad: N+1 queries
# {% for article in articles %}
#     {{ article.author.name }}  <!-- Query per article -->
# {% endfor %}

# Good: Use select_related in view
def article_list(request):
    articles = Article.objects.select_related('author').all()
    return render(request, 'articles.html', {'articles': articles})
```

#### 2. Cache Expensive Operations
**Description**: Cache the results of expensive computations to avoid recalculating them on every request.

**When to use**: For operations that are computationally expensive but don't change frequently.

```python
# Custom tag with caching
from django.core.cache import cache

@register.simple_tag
def expensive_calculation(param):
    cache_key = f'expensive_calc_{param}'
    result = cache.get(cache_key)
    
    if result is None:
        result = perform_expensive_calculation(param)
        cache.set(cache_key, result, 3600)  # Cache for 1 hour
    
    return result

# Template fragment caching
# {% load cache %}
# {% cache 300 sidebar %}
#     {% for category in categories %}
#         <li>{{ category.name }}</li>
#     {% endfor %}
# {% endcache %}
```

#### 3. Use Efficient Filters
**Description**: Avoid applying complex filters within loops, as they get executed for every iteration.

**When to optimize**: Move filter applications to the view when they would be called repeatedly in templates.

```python
# Avoid complex filters in loops
# Bad:
# {% for item in items %}
#     {{ item|complex_filter|another_filter }}
# {% endfor %}

# Good: Pre-process in view
def my_view(request):
    items = []
    for item in raw_items:
        processed_item = complex_filter(another_filter(item))
        items.append(processed_item)
    
    return render(request, 'template.html', {'items': items})
```

### Memory Usage Optimization

**Description**: Memory optimization prevents excessive memory consumption, especially when dealing with large datasets or high-traffic applications.

**Key strategies**: Use pagination, iterators, and avoid loading large datasets into memory unnecessarily.

#### 1. Avoid Large Context Data
**Description**: Passing large querysets or datasets to templates can consume significant memory and slow down rendering.

**When to use**: Always paginate large datasets and only pass the data that will actually be displayed.

```python
# Bad: Pass entire querysets
def article_list(request):
    all_articles = Article.objects.all()  # Could be millions
    return render(request, 'list.html', {'articles': all_articles})

# Good: Use pagination
from django.core.paginator import Paginator

def article_list(request):
    articles = Article.objects.all()
    paginator = Paginator(articles, 25)  # 25 per page
    page = request.GET.get('page')
    articles_page = paginator.get_page(page)
    
    return render(request, 'list.html', {
        'articles': articles_page,
        'paginator': paginator
    })
```

#### 2. Use Iterators for Large Datasets
**Description**: Iterators load objects one at a time instead of loading the entire queryset into memory.

**When to use**: For processing large datasets where you don't need all objects in memory simultaneously.

```python
# For very large exports
def export_data(request):
    response = StreamingHttpResponse(
        generate_csv_rows(),
        content_type='text/csv'
    )
    response['Content-Disposition'] = 'attachment; filename="data.csv"'
    return response

def generate_csv_rows():
    yield 'id,name,email\n'
    for user in User.objects.iterator():  # Memory efficient
        yield f'{user.id},{user.name},{user.email}\n'
```

### Template Loading Optimization

**Description**: Template loading optimization reduces the overhead of finding and parsing template files.

**Key strategies**: Use cached loaders and minimize the number of template includes.

#### 1. Use Cached Template Loaders
**Description**: Cached template loaders store compiled templates in memory to avoid re-parsing on every request.

**When to use**: In production environments where template files don't change frequently.

```python
# settings.py
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': ['templates'],
        'APP_DIRS': True,
        'OPTIONS': {
            'loaders': [
                ('django.template.loaders.cached.Loader', [
                    'django.template.loaders.filesystem.Loader',
                    'django.template.loaders.app_directories.Loader',
                ]),
            ],
        },
    },
]
```

#### 2. Minimize Template Includes
**Description**: Each template include adds overhead for file loading and parsing.

**When to optimize**: Use template inheritance instead of multiple includes for common page structures.

```django
<!-- Avoid too many includes -->
{% include "header.html" %}
{% include "nav.html" %}
{% include "sidebar.html" %}
{% include "content.html" %}
{% include "footer.html" %}

<!-- Better: Use template inheritance -->
{% extends "base.html" %}

{% block nav %}
    <!-- Navigation content -->
{% endblock %}

{% block content %}
    <!-- Main content -->
{% endblock %}
```

### Monitoring Template Performance

**Description**: Monitoring helps identify performance bottlenecks and track the effectiveness of optimizations.

**Key strategies**: Use debugging tools, custom profiling, and automated monitoring.

#### 1. Django Debug Toolbar
**Description**: Django Debug Toolbar provides detailed information about template rendering time, SQL queries, and other performance metrics.

**When to use**: During development and staging to identify performance issues.

```python
# settings.py
INSTALLED_APPS = [
    # ...
    'debug_toolbar',
]

MIDDLEWARE = [
    # ...
    'debug_toolbar.middleware.DebugToolbarMiddleware',
]

DEBUG_TOOLBAR_CONFIG = {
    'SHOW_TEMPLATE_CONTEXT': True,
}
```

#### 2. Custom Template Timing
**Description**: Custom profiling tags allow you to measure the rendering time of specific template blocks.

**When to use**: For detailed performance analysis of specific template sections.

```python
# templatetags/profile_tags.py
import time
from django.template import Node

class ProfileNode(Node):
    def __init__(self, nodelist, name):
        self.nodelist = nodelist
        self.name = name
    
    def render(self, context):
        start = time.time()
        output = self.nodelist.render(context)
        end = time.time()
        
        print(f"Template block '{self.name}' took {end - start:.4f}s")
        return output

@register.tag
def profile(parser, token):
    """Profile template rendering time: {% profile "block_name" %}...{% endprofile %}"""
    bits = token.split_contents()
    if len(bits) != 2:
        raise template.TemplateSyntaxError(
            "profile tag requires one argument: the block name"
        )
    
    name = bits[1].strip('"')
    nodelist = parser.parse(('endprofile',))
    parser.delete_first_token()
    return ProfileNode(nodelist, name)

# Usage
# {% profile "sidebar" %}
#     <!-- Expensive sidebar content -->
# {% endprofile %}
```

#### 3. Template Rendering Metrics
**Description**: Middleware can track overall request performance including template rendering time and database query counts.

**When to use**: For production monitoring and alerting on slow requests.

```python
# middleware.py
import time
from django.db import connection

class TemplatePerformanceMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response
    
    def __call__(self, request):
        start_time = time.time()
        start_queries = len(connection.queries)
        
        response = self.get_response(request)
        
        end_time = time.time()
        end_queries = len(connection.queries)
        
        # Log performance metrics
        total_time = end_time - start_time
        query_count = end_queries - start_queries
        
        if total_time > 1.0:  # Log slow requests
            print(f"Slow request: {request.path} took {total_time:.2f}s, {query_count} queries")
        
        return response
```

### Common Performance Issues

**Description**: These are the most frequent performance problems encountered with Django templates and their solutions.

#### 1. N+1 Query Problems
**Description**: Occurs when accessing related objects in loops without pre-loading, causing one database query per item.

**Solution**: Use `select_related` and `prefetch_related` in views to load related data efficiently.

```python
# Problematic template
# {% for article in articles %}
#     <div>{{ article.title }}</div>
#     <div>{{ article.author.name }}</div>  <!-- N+1 queries -->
# {% endfor %}

# Solution: Use select_related in view
articles = Article.objects.select_related('author').all()
```

#### 2. Expensive Filters in Loops
**Description**: Complex filters applied within loops get executed for every iteration, multiplying the computational cost.

**Solution**: Pre-process data in views or use template fragment caching.

```python
# Bad: Expensive filter called for each item
# {% for item in items %}
#     {{ item|expensive_filter }}
# {% endfor %}

# Good: Pre-compute in view
processed_items = [expensive_filter(item) for item in items]
context = {'items': processed_items}
```

#### 3. Large Template Context
```python
# Bad: Pass all data
context = {
    'all_users': User.objects.all(),  # Could be 100k users
    'all_articles': Article.objects.all(),
}

# Good: Pass only needed data
context = {
    'recent_users': User.objects.all()[:10],
    'featured_articles': Article.objects.filter(featured=True)[:5],
}
```

### Caching Strategies

#### 1. Template Fragment Caching
```django
{% load cache %}

<!-- Cache entire sidebar for 1 hour -->
{% cache 3600 sidebar request.user.pk %}
    <div class="sidebar">
        {% for category in categories %}
            <a href="{% url 'category' category.slug %}">{{ category.name }}</a>
        {% endfor %}
    </div>
{% endcache %}
```

#### 2. Per-User Caching
```django
{% cache 1800 user_menu request.user.pk %}
    <nav class="user-menu">
        <a href="{% url 'profile' %}">Profile</a>
        <a href="{% url 'settings' %}">Settings</a>
        {% if request.user.is_staff %}
            <a href="{% url 'admin' %}">Admin</a>
        {% endif %}
    </nav>
{% endcache %}
```

#### 3. API Response Caching
```python
# views.py
from django.views.decorators.cache import cache_page

@cache_page(60 * 15)  # Cache for 15 minutes
def api_endpoint(request):
    data = get_expensive_data()
    return JsonResponse(data)
```

### Database Optimization

#### 1. Use select_related and prefetch_related
```python
def article_detail(request, slug):
    article = Article.objects.select_related(
        'author',
        'category'
    ).prefetch_related(
        'tags',
        'comments__author'
    ).get(slug=slug)
    
    return render(request, 'article_detail.html', {
        'article': article
    })
```

#### 2. Defer Heavy Fields
```python
def article_list(request):
    # Defer large text fields for list view
    articles = Article.objects.defer('content').select_related('author')
    return render(request, 'article_list.html', {'articles': articles})
```

### Monitoring and Alerting

#### 1. Slow Query Logging
```python
# settings.py
LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'handlers': {
        'slow_query_file': {
            'level': 'WARNING',
            'class': 'logging.FileHandler',
            'filename': 'logs/slow_queries.log',
        },
    },
    'loggers': {
        'django.db.backends': {
            'handlers': ['slow_query_file'],
            'level': 'WARNING',
            'filters': [
                {
                    '()': 'myapp.logging.SlowQueryFilter',
                },
            ],
        },
    },
}
```

#### 2. Performance Metrics
```python
# metrics.py
from django.core.signals import request_finished
from django.dispatch import receiver
import time

rendering_times = []

@receiver(request_finished)
def log_rendering_time(sender, **kwargs):
    # This is a simplified example
    # In production, use proper metrics collection
    if hasattr(sender, '_start_time'):
        duration = time.time() - sender._start_time
        rendering_times.append(duration)
        
        if duration > 2.0:  # Alert on slow renders
            print(f"Slow render: {duration:.2f}s for {sender.path}")
```

### Scaling Considerations

#### 1. CDN for Static Assets
```python
# settings.py
AWS_S3_CUSTOM_DOMAIN = 'cdn.example.com'
STATIC_URL = f'https://{AWS_S3_CUSTOM_DOMAIN}/'
```

#### 2. Template Precompilation
```python
# For high-traffic sites, precompile templates
from django.template import engines
from django.template.engine import Engine

def precompile_templates():
    engine = engines['django']
    # Preload commonly used templates
    engine.get_template('base.html')
    engine.get_template('article_detail.html')
    # ... other templates
```

#### 3. Database Connection Pooling
```python
# settings.py
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'CONN_MAX_AGE': 60,  # Keep connections alive for 60 seconds
        'OPTIONS': {
            'CONN_MAX_AGE': 60,
        },
    }
}
```